In [9]:
# Bloco 1: Instalações
%pip -q install google-genai
!pip install -q google-adk
!pip install -q python-docx

# Bloco 2: Configuração da API Key
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Bloco 3: Configuração do Cliente Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Bloco 4: Importar display
from IPython.display import HTML, Markdown

# Bloco 5: Importações ADK e outras
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display
import warnings
import ipywidgets as widgets
import asyncio
from docx import Document
from docx.shared import Inches
import base64

warnings.filterwarnings("ignore")

# Bloco 6: Função auxiliar call_agent
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
            break
    return final_response

# Bloco 7: Função auxiliar to_markdown (ainda útil para formatação interna)
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Bloco 8: Agente Planejador
def agente_planejador(informacoes_usuario):
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        description="""Recebe e analisa as informações inseridas pelo usuário para planejar lançamentos musicais, turnês e outros projetos.
          Define etapas, prazos e busca informações relevantes usando o Google.""",
        tools=[google_search],
        instruction=f"""
        Você é um planejador estratégico especializado na carreira de artistas musicais. Sua tarefa é analisar as informações fornecidas pelo usuário
        para criar um plano de ação detalhado para o objetivo musical especificado (lançamento de single, EP, álbum, turnê, etc.).

        Mantenha o plano claro, conciso e prático, com etapas bem definidas e prazos sugeridos.
        """,
    )
    plano_inicial = call_agent(planejador, f"Gere um plano de lançamento detalhado com base nas seguintes informações: {informacoes_usuario}")
    return plano_inicial

# Bloco 9: Agente de Marketing
def agente_de_marketing(plano_de_lancamento, data_de_lancamento):
    marketing_agente = Agent(
        name="agente_de_marketing",
        model=MODEL_ID,
        description="""Recebe o plano de lançamento e a data de lançamento para desenvolver um cronograma de conteúdo para Instagram, TikTok e YouTube,
        sugerindo frequência, tipos de conteúdo e práticas estratégicas.""",
        instruction="""
        Você é um especialista em marketing digital focado em artistas musicais, com experiência nas plataformas Instagram, TikTok e YouTube.
        Sua tarefa é analisar o plano de lançamento fornecido e a data de lançamento para criar um cronograma de conteúdo estratégico para as redes sociais.

        O cronograma deve ser claro, organizado por plataforma e fase do lançamento, e conter sugestões práticas e acionáveis.
        """,
    )
    entrada_do_agente_de_marketing = f"""Plano de Lançamento: {plano_de_lancamento}
Data de Lançamento: {data_de_lancamento}"""
    cronograma_marketing = call_agent(marketing_agente, entrada_do_agente_de_marketing)
    return cronograma_marketing

# Bloco 10: Agente Financeiro
def agente_financeiro(objetivo_lancamento, orcamento_disponivel, cronograma_marketing, plano_lancamento):
    financeiro_agente = Agent(
        name="agente_financeiro",
        model=MODEL_ID,
        description="""Recebe o objetivo de lançamento, o orçamento disponível, o cronograma de marketing e o plano de lançamento para estimar custos e criar uma organização financeira em formato Markdown, evitando o uso do símbolo $.""",
        tools=[google_search],
        instruction=f"""
        Você é um especialista em gestão financeira para artistas musicais. Sua tarefa é analisar o objetivo de lançamento:
        "{objetivo_lancamento}", o orçamento disponível: "{orcamento_disponivel}", o cronograma de marketing:
        "{cronograma_marketing}" e o plano de lançamento: "{plano_lancamento}" para estimar os custos envolvidos em cada etapa
        e criar uma organização financeira clara e bem formatada em Markdown, descrevendo os valores monetários textualmente.

        Apresente a estimativa de custos em uma lista Markdown, detalhando o custo estimado para cada etapa, indicando os valores com a palavra "reais" (ex: mil e quinhentos reais).

        Após a estimativa, sugira uma alocação do orçamento disponível para cada etapa em outra lista Markdown, seguindo o mesmo formato para os valores.

        Informe sobre possíveis restrições orçamentárias e sugira alternativas ou priorizações de gastos, se necessário, em um parágrafo conciso, também evitando o símbolo "R$".

        Mantenha a formatação limpa e utilize Markdown para títulos, listas e ênfase.
        """,
    )
    entrada_do_agente_financeiro = f"""Objetivo do Lançamento: {objetivo_lancamento}
Orçamento Disponível: {orcamento_disponivel}
Cronograma de Marketing: {cronograma_marketing}
Plano de Lançamento: {plano_lancamento}"""
    plano_financeiro = call_agent(financeiro_agente, entrada_do_agente_financeiro)
    return plano_financeiro

# Bloco 11: Agente Organizador
def agente_organizador(plano_de_lancamento, cronograma_de_marketing, plano_financeiro):
    organizador_agente = Agent(
        name="agente_organizador",
        model=MODEL_ID,
        description="""Recebe os planos de lançamento, marketing e financeiro para organizar tarefas em um cronograma unificado e gerar listas de verificação.""",
        tools=[google_search],
        instruction="""
        Você é um assistente de organização de projetos musicais. Sua tarefa é receber o plano de lançamento, o cronograma de marketing
        e o plano financeiro e integrá-los em um cronograma unificado de tarefas com prazos e dependências. Além disso, gere listas de
        verificação para cada etapa principal do plano de lançamento.

        Com base no Plano de Lançamento, Cronograma de Marketing e Plano Financeiro fornecidos, identifique todas as tarefas acionáveis.
        Organize essas tarefas em um cronograma sequencial, atribuindo prazos realistas a cada uma, considerando as dependências entre elas.

        Para cada etapa principal do Plano de Lançamento, crie uma lista de verificação dos itens essenciais a serem concluídos, utilizando a seguinte formatação para as caixas de seleção:
        - `[ ]` para itens não marcados
        - `[x]` para itens marcados (o agente não precisa marcar nada inicialmente, apenas usar a formatação)
        """,
    )
    entrada_do_agente_organizador = f"""Plano de Lançamento: {plano_de_lancamento}
Cronograma de Marketing: {cronograma_de_marketing}
Plano Financeiro: {plano_financeiro}"""
    cronograma_unificado = call_agent(organizador_agente, entrada_do_agente_organizador)
    return cronograma_unificado

# Bloco 12: Agente Mestre
def agente_mestre(informacoes_usuario):
    mestre = Agent(
        name="agente_mestre",
        model=MODEL_ID,
        description="""Recebe a solicitação do usuário e coordena a interação entre os agentes de planejamento, marketing, financeiro e organizador para gerar um plano de lançamento integrado.""",
        instruction="""
        Você é o maestro da gestão de lançamentos musicais. Sua tarefa é orquestrar a interação entre os agentes para gerar um plano de lançamento completo e integrado.

        Siga os seguintes passos:

        1.  **Planejamento Inicial:** Chame o `agente_planejador`.
        2.  **Planejamento de Marketing:** Chame o `agente_de_marketing`.
        3.  **Planejamento Financeiro:** Chame o `agente_financeiro`.
        4.  **Organização:** Chame o `agente_organizador`.
        5.  **Apresentação:** Combine as respostas de todos os agentes em um plano de lançamento integrado.
        """,
        tools=[]
    )
    print("AGENTE MESTRE: Iniciando o planejamento inicial...")
    plano_lancamento = agente_planejador(informacoes_usuario)
    print("AGENTE MESTRE: Planejamento inicial concluído.")
    print("\nAGENTE MESTRE: Iniciando o planejamento de marketing...")
    cronograma_marketing = agente_de_marketing(plano_lancamento, informacoes_usuario['Data prevista'])
    print("AGENTE MESTRE: Planejamento de marketing concluído.")
    print("\nAGENTE MESTRE: Iniciando o planejamento financeiro...")
    plano_financeiro = agente_financeiro(informacoes_usuario['Objetivo principal'], informacoes_usuario['Orçamento disponível'], cronograma_marketing, plano_lancamento)
    print("AGENTE MESTRE: Planejamento financeiro concluído.")
    print("\nAGENTE MESTRE: Iniciando a organização...")
    cronograma_unificado = agente_organizador(plano_lancamento, cronograma_marketing, plano_financeiro)
    print("AGENTE MESTRE: Organização concluída.")
    return f"""## Plano de Lançamento Integrado

### Plano de Lançamento Inicial:
{plano_lancamento}

### Cronograma de Marketing:
{cronograma_marketing}

### Plano Financeiro:
{plano_financeiro}

### Cronograma Unificado e Listas de Verificação:
{cronograma_unificado}
"""

# Bloco 13: Definição e Exibição do Formulário e Função Gerar Plano Final com DOCX
import ipywidgets as widgets
from IPython.display import display, HTML
from docx import Document
from docx.shared import Inches
import base64

# Widgets do formulário
nome_artista = widgets.Text(description="Nome do Artista/Banda:")
objetivo_principal = widgets.Text(description="Objetivo Principal:")
data_prevista = widgets.Text(description="Data Prevista:")
orcamento_disponivel = widgets.Text(description="Orçamento Disponível:")
local_execucao = widgets.Text(description="Local de Execução:")
genero_musical = widgets.Text(description="Gênero Musical:")
estagio_projeto = widgets.Dropdown(
    description="Estágio do Projeto:",
    options=['0 - 3 anos', '3 - 7 anos', '7 - 15 anos', 'mais de 15 anos'],
    value='0 - 3 anos'
)
outras_informacoes = widgets.Textarea(description="Outras Informações - ex: release do projeto:")
botao_enviar = widgets.Button(description="Gerar Plano (DOCX)")
saida_processamento = widgets.Output()

def gerar_link_download_docx(docx_document, filename="plano_integrado.docx"):
    """Gera um link para download de um objeto Document do python-docx."""
    from io import BytesIO
    f = BytesIO()
    docx_document.save(f)
    b64 = base64.b64encode(f.getvalue()).decode()
    href = f'<a href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}" download="{filename}">Clique aqui para baixar o plano integrado (formato DOCX)</a>'
    return href

def gerar_plano_final_docx(event):
    print("A função gerar_plano foi chamada!")
    informacoes_usuario_formulario = {
        'Nome do Artista/Banda': nome_artista.value,
        'Objetivo principal': objetivo_principal.value,
        'Data prevista': data_prevista.value,
        'Orçamento disponível': orcamento_disponivel.value,
        'Local de execução': local_execucao.value,
        'Gênero musical': genero_musical.value,
        'Estágio do projeto': estagio_projeto.value,
        'Outras informações - ex: release do projeto': outras_informacoes.value
    }
    print("Informações do usuário coletadas:")
    print(informacoes_usuario_formulario)

    # Chama o agente MESTRE de forma síncrona
    plano_integrado_markdown = agente_mestre(informacoes_usuario_formulario)

    # Cria um documento Word
    document = Document()
    document.add_heading('Plano de Lançamento Integrado', level=0)

    def add_markdown_to_docx(markdown_text, document):
        for line in markdown_text.splitlines():
            if line.startswith("###"):
                document.add_heading(line[3:].strip(), level=1)
            elif line.startswith("**"):
                paragraph = document.add_paragraph()
                paragraph.add_run(line[2:-2].strip()).bold = True
            elif line.startswith("* "):
                document.add_paragraph(line[2:].strip(), style='List Bullet')
            elif line.startswith("[ ] "):
                paragraph = document.add_paragraph()
                paragraph.add_run('☐ ').bold = False # Caixa de seleção vazia (Unicode U+2610)
                paragraph.add_run(line[4:].strip()).bold = False
            elif line.startswith("[x] "):
                paragraph = document.add_paragraph()
                paragraph.add_run('☑ ').bold = False # Caixa de seleção marcada (Unicode U+2611)
                paragraph.add_run(line[4:].strip()).bold = False
            else:
                document.add_paragraph(line)

    # Adiciona o conteúdo Markdown ao documento Word linha por linha
    for section in plano_integrado_markdown.split('###'):
        if section.strip():
            heading = section.split('\n')[0].strip()
            content = '\n'.join(section.split('\n')[1:]).strip()
            if heading:
                document.add_heading(heading, level=1)
            if content:
                add_markdown_to_docx(content, document)

    # Gera o link para download do DOCX
    link_download = gerar_link_download_docx(document)
    display(HTML(link_download))

# Exibição do formulário e associação do evento (agora com o botão DOCX)
display(nome_artista, objetivo_principal, data_prevista, orcamento_disponivel, local_execucao,
        genero_musical, estagio_projeto, outras_informacoes, botao_enviar, saida_processamento)
botao_enviar.on_click(gerar_plano_final_docx)

# Bloco 14: Agente Mestre
def agente_mestre(informacoes_usuario):
    mestre = Agent(
        name="agente_mestre",
        model=MODEL_ID,
        description="""Recebe a solicitação do usuário e coordena a interação entre os agentes de planejamento, marketing, financeiro e organizador para gerar um plano de lançamento integrado.""",
        instruction="""
        Você é o maestro da gestão de lançamentos musicais. Sua tarefa é orquestrar a interação entre os agentes para gerar um plano de lançamento completo e integrado.

        Siga os seguintes passos:

        1.  **Planejamento Inicial:** Chame o `agente_planejador`.
        2.  **Planejamento de Marketing:** Chame o `agente_de_marketing`.
        3.  **Planejamento Financeiro:** Chame o `agente_financeiro`.
        4.  **Organização:** Chame o `agente_organizador`.
        5.  **Apresentação:** Combine as respostas de todos os agentes em um plano de lançamento integrado.
        """,
        tools=[]
    )
    print("AGENTE MESTRE: Iniciando o planejamento inicial...")
    plano_lancamento = agente_planejador(informacoes_usuario)
    print("AGENTE MESTRE: Planejamento inicial concluído.")
    print("\nAGENTE MESTRE: Iniciando o planejamento de marketing...")
    cronograma_marketing = agente_de_marketing(plano_lancamento, informacoes_usuario['Data prevista'])
    print("AGENTE MESTRE: Planejamento de marketing concluído.")
    print("\nAGENTE MESTRE: Iniciando o planejamento financeiro...")
    plano_financeiro = agente_financeiro(informacoes_usuario['Objetivo principal'], informacoes_usuario['Orçamento disponível'], cronograma_marketing, plano_lancamento)
    print("AGENTE MESTRE: Planejamento financeiro concluído.")
    print("\nAGENTE MESTRE: Iniciando a organização...")
    cronograma_unificado = agente_organizador(plano_lancamento, cronograma_marketing, plano_financeiro)
    print("AGENTE MESTRE: Organização concluída.")
    return f"""## Plano de Lançamento Integrado

### Plano de Lançamento Inicial:
{plano_lancamento}

### Cronograma de Marketing:
{cronograma_marketing}

### Plano Financeiro:
{plano_financeiro}

### Cronograma Unificado e Listas de Verificação:
{cronograma_unificado}
"""

Text(value='', description='Nome do Artista/Banda:')

Text(value='', description='Objetivo Principal:')

Text(value='', description='Data Prevista:')

Text(value='', description='Orçamento Disponível:')

Text(value='', description='Local de Execução:')

Text(value='', description='Gênero Musical:')

Dropdown(description='Estágio do Projeto:', options=('0 - 3 anos', '3 - 7 anos', '7 - 15 anos', 'mais de 15 an…

Textarea(value='', description='Outras Informações - ex: release do projeto:')

Button(description='Gerar Plano (DOCX)', style=ButtonStyle())

Output()